## Summary

---

## Imports

In [1]:
import gzip
import subprocess
import threading
import time
import uuid
from enum import Enum
from pathlib import Path

import ipywidgets as widgets
import numpy as np
from IPython.display import HTML, Javascript, display

## Parameters

In [2]:
JUPYTER_DATA_DIR = Path(subprocess.check_output(["jupyter", "--data-dir"], universal_newlines=True).strip()).resolve(strict=True)

In [3]:
JUPYTER_DATA_DIR

PosixPath('/home/kimlab1/strokach/.jupyter/data')

In [4]:
STATIC_DATA_DIR = JUPYTER_DATA_DIR.joinpath("voila", "templates", "mytemplate", "static").resolve(strict=True)

In [5]:
STATIC_DATA_DIR

PosixPath('/home/kimlab1/strokach/.jupyter/data/voila/templates/mytemplate/static')

## Define global variables

In [6]:
sequences = []
proteinsolver_thread = None

## Helper functions

In [7]:
class ProteinSolverThread(threading.Thread):
    def __init__(self, value, msa_view, progress_bar, download_button):
        super().__init__(daemon=True)
        self.value = value
        self.msa_view = msa_view
        self.msa_view.clear_output()
        self.progress_bar = progress_bar
        self.download_button = download_button
        self._stop_event = threading.Event()

    def run(self):
        self.progress_bar.value = 0
        self.progress_bar.bar_style = ""
        for i in range(self.progress_bar.max):
            if self.stopped():
                self.progress_bar.bar_style = "danger"
                return
            time.sleep(1)
            self.msa_view.append_stdout(self.value + "\n")
            self.progress_bar.value += 1
        self.progress_bar.bar_style = "success"
        enable_download_button(self.download_button)

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()

In [8]:
def generate_random_sequence(length=80, seed=None):
    amino_acids = np.array(list("GVALICMFWPDESTYQNKRH"))
    if seed is None:
        choice = np.random.choice
    else:
        choice = np.random.RandomState(seed).choice
    return "".join(choice(amino_acids, length))

In [9]:
generate_random_sequence(80, 42)

'MHYDFMRDDLFAVECVGEENPQYYREHAIRMWMKLTKWVHYMEFYATNLKFLVCPLKEVPLTQYFTFQSKYSWYSGMWGE'

In [10]:
def sequences_to_fasta(sequences, line_width=80):
    sequence_string = ""
    for sequence in sequences:
        sequence_string += f">{sequence['id']}|{sequence['name']}|{sequence['proba']}\n"
        for start in range(0, len(sequence["seq"]), line_width):
            sequence_string += sequence["seq"][start : start + line_width] + "\n"
    return sequence_string

In [11]:
print(sequences_to_fasta([{"id": 1, "name": "reference", "proba": 1.0, "seq": generate_random_sequence(160, 42)}]))

>1|reference|1.0
MHYDFMRDDLFAVECVGEENPQYYREHAIRMWMKLTKWVHYMEFYATNLKFLVCPLKEVPLTQYFTFQSKYSWYSGMWGE
FDRNFAAGIPMWMWFEVGQIAEFAGAIYTAGITMWYYPSRMNHLIMSYDLSMRVPSCEEHDMGGHSWAMCFWIGRPEYWH



In [12]:
def populate_sequences_list():
    global sequences
    
    sequences = []
    for i in range(20_000):
        sequence = {"id": i + 1, "name": f"gen-{i:05d}", "proba": 1.0, "seq": generate_random_sequence(162)}
        sequences.append(sequence)

In [13]:
populate_sequences_list()

In [14]:
def save_sequences():
    sequences_fasta = sequences_to_fasta(sequences)
    sequences_fasta_gz = gzip.compress(sequences_fasta.encode("utf-8"))

    output_file = STATIC_DATA_DIR.joinpath(f"{uuid.uuid4()}.fasta.gz")
    with output_file.open("wb") as fout:
        fout.write(sequences_fasta_gz)

    return output_file

In [15]:
def show_uploader():
    uploader = widgets.FileUpload(accept='.gpx', multiple=False)

    def handle_upload(change):
        # keep only the last file
        # TODO: check if this should be fixed in FileUpload widget
        # when multiple=False
        *_, (_, f) = change['new'].items()
        gpx_content = f['content'].decode('utf-8')
        out.clear_output()
        with StringIO(gpx_content) as gpx_file:
            with out:
                plot_gpx(gpx_file)

    uploader.observe(handle_upload, names='value')

    display(uploader)

In [16]:
def show_examples():
    example_folder = "./examples"
    examples = [f for f in os.listdir(example_folder) if f.endswith('.gpx')]
    
    def create_example(name):
        filename = os.path.join(example_folder, name)
        
        @out.capture()
        def on_example_clicked(change):
            out.clear_output()
            with open(filename) as f:
                with out:
                    plot_gpx(f)
    
        button = Button(description=os.path.splitext(name)[0])
        button.on_click(on_example_clicked)
        return button

    
    buttons = [create_example(example) for example in examples]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)

## Widgets

In [17]:
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    bar_style="",  # 'success', 'info', 'warning', 'danger' or ''
    orientation="horizontal",
    layout=widgets.Layout(width="auto", height="20px"),
)

In [18]:
msa_alignment_view = widgets.Output()

In [19]:
number_of_sequences_input = widgets.BoundedIntText(
    value=100,
    min=1,
    max=20_000,
    step=1,
    description="Number of sequences:",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="200px"),
)

In [20]:
class State(Enum):
    GENERATE: str = "Run ProteinSolver!"
    CANCEL: str = "Cancel"


button_meta = {
    State.GENERATE: {"icon": "check", "button_style": "", "tooltip": "Generate new sequences!"},
    State.CANCEL: {"icon": "ban", "button_style": "danger", "tooltip": "Cancel!"},
}


def disable_download_button(b):
    b.description = State.CANCEL.value
    b.icon = button_meta[State.CANCEL]["icon"]
    b.button_style = button_meta[State.CANCEL]["button_style"]
    b.tooltip = button_meta[State.CANCEL]["tooltip"]


def enable_download_button(b):
    b.description = State.GENERATE.value
    b.icon = button_meta[State.GENERATE]["icon"]
    b.button_style = button_meta[State.GENERATE]["button_style"]
    b.tooltip = button_meta[State.GENERATE]["tooltip"]


def on_button_clicked(b):
    global proteinsolver_thread

    if b.description == State.GENERATE.value:
        disable_download_button(b)

        if proteinsolver_thread is not None and not proteinsolver_thread.stopped():
            proteinsolver_thread.stop()
        progress_bar.max = number_of_sequences_input.value
        proteinsolver_thread = ProteinSolverThread("hello world", msa_alignment_view, progress_bar, b)
        proteinsolver_thread.start()
    elif b.description == State.CANCEL.value:
        enable_download_button(b)

        proteinsolver_thread.stop()
    else:
        raise Exception


run_proteinsolver_button = widgets.Button(
    description=State.GENERATE.value,
    icon=button_meta[State.GENERATE]["icon"],
    button_style=button_meta[State.GENERATE]["button_style"],  # 'success', 'info', 'warning', 'danger' or ''
    tooltip=button_meta[State.GENERATE]["tooltip"],
    disabled=False,
    layout=widgets.Layout(width="200px"),
)
run_proteinsolver_button.on_click(on_button_clicked)

In [21]:
widgets.VBox([number_of_sequences_input, run_proteinsolver_button])

In [22]:
generate_download_link_output = widgets.Output(layout=widgets.Layout(width="200px"))

In [23]:
def generate_download_link(b):
    b.description = "Generating..."
    b.icon = "running"
    b.button_style = "info"  # 'success', 'info', 'warning', 'danger' or ''
    b.disabled = True

    generate_download_link_output.clear_output()
    output_file = save_sequences()
    with generate_download_link_output:
        display(
            HTML(
                f'<a href="./voila/static/{output_file.name}" download={output_file.stem[:8]}{output_file.suffix}><i class="fa fa-download"></i> Download sequences</a>'
            )
        )

    b.description = "Update download link"
    b.icon = ""  # check
    b.button_style = ""
    b.disabled = False


generate_download_link_button = widgets.Button(
    description="Generate download link",
    tooltip="Generate download link",
    disabled=False,
    layout=widgets.Layout(width="200px"),
)

generate_download_link_button.on_click(generate_download_link)

In [24]:
widgets.VBox([generate_download_link_button, generate_download_link_output])

## Final dashboard

In [25]:
grid = widgets.GridspecLayout(5, 3, height='400px')

grid[0, :] = progress_bar
grid[1, 0] = widgets.VBox([number_of_sequences_input, run_proteinsolver_button])
grid[2, 0] = widgets.VBox([generate_download_link_button, generate_download_link_output])
grid[1:, 1:] = msa_alignment_view

grid

GridspecLayout(children=(IntProgress(value=0, layout=Layout(grid_area='widget001', height='20px', width='auto'…

In [26]:
on_button_clicked(run_proteinsolver_button)